In [1]:
## Demo Notebook example
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

import pandas as pd
%pprint

ModuleNotFoundError: No module named 'ortools'

In [ ]:
### this is yff morning durations where two vehicles will be utilized , no pick up schedule provided 
### the problem is to utilize the two vehicles with the minimum duration possible 
import pandas as pd
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
dist=pd.read_excel('input/durations.xlsx')
window=pd.read_excel('input/time_windows.xlsx')

window

In [ ]:
ab=[]
for i in range(0,dist.shape[0]):
   a=dist.loc[i,:].values
   a=a.tolist()
   ab.append(a)
ab

In [ ]:
data={}
data['time_matrix']=ab
data['num_vehicles'] = 2
data['depot'] = 0
data['time_windows']=[]
for i in range(0,window.shape[0]):
    data['time_windows'].append((window['a'][i],window['b'][i]))


In [ ]:
len(data['time_matrix'])


In [ ]:
len(data['time_windows'])

In [ ]:
data['time_windows']


In [ ]:
manager= pywrapcp.RoutingIndexManager(len(data['time_matrix']),data['num_vehicles'],data['depot'])

routing= pywrapcp.RoutingModel(manager)


In [ ]:
def time_callback(from_index,to_index):
    from_node= manager.IndexToNode(from_index)
    to_node= manager.IndexToNode(to_index)
    return data['time_matrix'][from_node][to_node]

transit_callback_index= routing.RegisterTransitCallback(time_callback)

routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

time='Time'
routing.AddDimension( transit_callback_index
    , 260
    , 260, False, time)




In [ ]:
time_dimension= routing.GetDimensionOrDie(time)
time_dimension.SetGlobalSpanCostCoefficient(1000)

data['time_windows'][0][0]

In [ ]:
###Adding time windows consttraint

for node, time_window in enumerate(data['time_windows']):
    if node ==0:
        continue
    index= manager.NodeToIndex(node)
    time_dimension.CumulVar(index).SetRange(int(time_window[0]),int(time_window[1]))




In [ ]:
for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        time_dimension.CumulVar(index).SetRange(int(data['time_windows'][0][0]),
                                                int(data['time_windows'][0][1]))

for i in range(data['num_vehicles']):
        routing.AddVariableMinimizedByFinalizer(
            time_dimension.CumulVar(routing.Start(i)))
        routing.AddVariableMinimizedByFinalizer(
            time_dimension.CumulVar(routing.End(i)))


In [ ]:
    # Setting first solution heuristic.

search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
search_parameters.local_search_metaheuristic = (
    routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
search_parameters.time_limit.seconds=300


In [ ]:
# Solve the problem.
solution = routing.SolveWithParameters(search_parameters)



In [ ]:
def get_routes(solution,routing,manager):
    routes=[]
    for node in range(routing.vehicles()):
        index= routing.Start(node)
        route= [manager.IndexToNode(index)]
        while not routing.IsEnd(index):
            index= solution.Value(routing.NextVar(index))
            route.append(manager.IndexToNode(index))
        routes.append(route)
    return(routes)
get_routes(solution,routing,manager)            
        
        
  

In [ ]:
def get_time_windows(solution,routing,manager,dimension):
    all_list=[]
    for vehicle in range(routing.vehicles()):
        route_data=[]
        index= routing.Start(vehicle)
        route= manager.IndexToNode(index)
        dim= dimension.CumulVar(index)
        route_data.append([route,solution.Min(dim),solution.Max(dim)])
        while not routing.IsEnd(index):
            index= solution.Value(routing.NextVar(index))
            route= manager.IndexToNode(index)
            dim= dimension.CumulVar(index)
            route_data.append([route,solution.Min(dim),solution.Max(dim)])
        all_list.append(route_data)
    return all_list


get_time_windows(solution,routing,manager,time_dimension) 